### **Создаем сервис для обработки запросов к модели**

In [10]:
import json 
import dill
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify

In [7]:
with open('rf_diabetes.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [8]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Запустить сервис и не глушить его, пока работаем 

In [17]:
# Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    age, gender, hypertension, heart_disease, smoking_history, bmi, HbA1c_level, blood_glucose_level = '','','','','','','',''
    
    request_json = request.get_json()
    
    if request_json["age"]:
        age = request_json['age']
        
    if request_json["gender"]:
        gender = request_json['gender']
                
    if request_json["hypertension"]:
        hypertension = request_json['hypertension']
        
    if request_json["heart_disease"]:
        heart_disease = request_json['heart_disease']
        
    if request_json["smoking_history"]:
        smoking_history = request_json['smoking_history']
        
    if request_json["bmi"]:
        bmi = request_json['bmi']
        
    if request_json["HbA1c_level"]:
        HbA1c_level = request_json['HbA1c_level']
        
    if request_json["blood_glucose_level"]:
        blood_glucose_level = request_json['blood_glucose_level']
     
    preds = model.predict_proba(pd.DataFrame({"age": [age],
                                              "gender": [gender],
                                              "hypertension": [hypertension],
                                              "heart_disease":[heart_disease],
                                              "smoking_history": [smoking_history],
                                              "bmi": [bmi],
                                              "HbA1c_level": [HbA1c_level],
                                              "blood_glucose_level": [blood_glucose_level]
                                             }))
    data["predictions"] = preds[:, 1][0]
   
        # indicate that the request was a success
    data["success"] = True
    print('OK')
    print(f'Предсказание: 1-наличие диабета, 0-отсутствие диабета')
        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-04-28 20:43:37,308] ERROR in app: Exception on /predict [POST]
TypeError: float() argument must be a string or a number, not 'list'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\ia_sv\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ia_sv\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\ia_sv\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\ia_sv\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\ia_sv\AppData\Local\Temp\ip

In [15]:
# Пример данных
age= '47'
gender= '1'
hypertension= '1' 
heart_disease= '0'
smoking_history= '0'
bmi= '26.3' 
HbA1c_level= '6'
blood_glucose_level = '85'



body = {
        'age': age,
        "gender": gender,
        'hypertension': hypertension, 
        'heart_disease': heart_disease,
        "smoking_history": smoking_history,
        'bmi': bmi,
        'HbA1c_level': HbA1c_level,
        'blood_glucose_level': blood_glucose_level
        }

In [16]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data

OK
Предсказание: 1-наличие диабета, 0-отсутствие диабета


{'predictions': 0.0, 'success': True}